In [1]:
from pathlib import Path
import pandas as pd

In [19]:
path_code_folder = Path('/home/fabian/projects/phd/angiosarkom_pdl1_prediction')
df_path = str(path_code_folder / 'data' / 'angiosarkom_patches_pdl1_mask_ANGIOSARCOM_for_real_new_annotations_all_pdl1_26sep23.feather')

In [ ]:
path_example_patch_df = '/home/fabian/projects/phd/APEDIA/data/example_patch_df.feather'
path_outputs = '/home/fabian/projects/phd/APEDIA/data/outputs'

In [20]:
df = pd.read_feather(df_path)

In [21]:
df = df[df.verified_angiosarcoma.eq(True)]

In [23]:
df_reduced = df[['path_patch_pdl1', 'path_patch_mask', 'cv_split', 'tumors']].reset_index()

In [24]:
# df_reduced.to_feather('/home/fabian/projects/phd/APEDIA/data/example_patch_df.feather')

In [15]:
df_reduced

,index,path_patch_pdl1,path_patch_mask,cv_split
0,0,/home/fabian/projects/phd/ssd_data/angiosarkom...,/home/fabian/projects/phd/ssd_data/angiosarkom...,1
1,1,/home/fabian/projects/phd/ssd_data/angiosarkom...,/home/fabian/projects/phd/ssd_data/angiosarkom...,1
2,2,/home/fabian/projects/phd/ssd_data/angiosarkom...,/home/fabian/projects/phd/ssd_data/angiosarkom...,1
3,3,/home/fabian/projects/phd/ssd_data/angiosarkom...,/home/fabian/projects/phd/ssd_data/angiosarkom...,1
4,4,/home/fabian/projects/phd/ssd_data/angiosarkom...,/home/fabian/projects/phd/ssd_data/angiosarkom...,1
...,...,...,...,...
28911,34012,/home/fabian/projects/phd/ssd_data/angiosarkom...,/home/fabian/projects/phd/ssd_data/angiosarkom...,0
28912,34013,/home/fabian/projects/phd/ssd_data/angiosarkom...,/home/fabian/projects/phd/ssd_data/angiosarkom...,0
28913,34014,/home/fabian/projects/phd/ssd_data/angiosarkom...,/home/fabian/projects/phd/ssd_data/angiosarkom...,0
28914,34015,/home/fabian/projects/phd/ssd_data/angiosarkom...,/home/fabian/projects/phd/ssd_data/angiosarkom...,0


In [17]:
from apedia.utils.params import train_tumor_patch_detector_params
train_tumor_patch_detector_params


{'column_name_data': 'pdl1',
 'column_name_mask': 'mask',
 'lr': 0.001,
 'epochs': 50,
 'bs': 8,
 'aug_mult': 1,
 'label_smoothing': 0.0,
 'encoder_name': 'timm-efficientnet-b5',
 'cv_split': 0,
 'num_workers': 4,
 'do_cosine_annealing': False}

# Same for segmentation

In [13]:
path_data_df = Path("/home/fabian/projects/phd/angiosarkom_pdl1_prediction/data/12oct23_hema_patch_if_more_matches_updated_roi_df_fixed_verified.feather")
df = pd.read_feather(path_data_df)

In [16]:
df['path_exact_one_match'].iloc[0]

'/home/fabian/projects/phd/angiosarkom_pdl1_prediction/data/cell_segmentation/21sep23_cell_segmentation_analysis_hema_patch_if_more_matches_updated_roi_df/calculation/W103-22_PD-L1__2336.0_2080.0_pdl1_segmentation_channels.npy'

In [4]:
res_df = df[df.verified_angiosarcoma.eq(True)].reset_index()
res_df['cv_split'] = res_df['cv']
res_df = res_df[['path_seg_one_match', 'path_patch_png', 'path_exact_one_match', 'cv_split']].reset_index(drop=True)
# res_df.to_feather("/home/fabian/projects/phd/APEDIA/data/example_df_segmentation.feather")

# Same for cell type data preprocessing

In [3]:
path_df = Path('/home/fabian/projects/phd/angiosarkom_pdl1_prediction') / 'data' / "pdl1_he_mask_patches_6mar23_tumor_non_tumor_cv_splits.feather"
df = pd.read_feather(path_df)

In [6]:
df.path_patch_pdl1.iloc[0]

'/home/fabian/projects/phd/ssd_data/angiosarkom_patches_pdl1_he_mask_6mar2023/pdl1/W103-22_PD-L1__2752.0_1856.0_pdl1.png'

In [7]:
# Old and new base paths
old_base_path = '/home/fabian/projects/phd/ssd_data'
new_base_path = '/home/fabian/raid5/formerly_on_ssd_data'

# Function to replace the base path
def replace_base_path(old_path, old_base, new_base):
    if old_path.startswith(old_base):
        return new_base + old_path[len(old_base):]
    return old_path

# Apply the function to each path in the DataFrame
df['path_patch_pdl1'] = df['path_patch_pdl1'].apply(lambda x: replace_base_path(x, old_base_path, new_base_path))

print(df['path_patch_pdl1'].iloc[0])

# df.to_feather('/home/fabian/projects/phd/APEDIA/data/example_cell_type_preprocess_path_df.feather')

/home/fabian/raid5/formerly_on_ssd_data/angiosarkom_patches_pdl1_he_mask_6mar2023/pdl1/W103-22_PD-L1__2752.0_1856.0_pdl1.png


### Create folder with all segmentation patches

In [6]:
from shutil import copy2
from tqdm.auto import tqdm

path_seg_patches = Path("/home/fabian/projects/phd/angiosarkom_pdl1_prediction/data/cell_segmentation/21sep23_cell_segmentation_analysis_hema_patch_if_more_matches_updated_roi_df/calculation/")
imgs = list(path_seg_patches.glob('*.png'))
target_folder = Path("/home/fabian/projects/phd/APEDIA/data/example_seg_patches/")
target_folder.mkdir(exist_ok=True)
# for img in tqdm(imgs):
#     copy2(img, target_folder)

100%|██████████| 508/508 [00:00<00:00, 1402.92it/s]
